In [1]:
import pandas as pd
df = pd.read_excel(r'C:\Users\prins\git\Human1_RPE_PR\RPE_PR data\expression_data.xlsx')
conditions = df.columns[2:]
df_discretized = df.copy(deep=True)

for c in conditions:
    q25 = df_discretized[c].quantile([0.25]).values[0]
    q75 = df_discretized[c].quantile([0.75]).values[0]
    df_discretized[c][df_discretized[c]<0.00001] = -1 # negative confidence (NC)
    df_discretized[c][(df_discretized[c]<=q25) & (df_discretized[c]>0)] = -10 # 1 'LC'
    df_discretized[c][(df_discretized[c]<=q75) & (df_discretized[c]>q25)] = -20 #  2 'MC'
    df_discretized[c][(df_discretized[c]>q75)] = -30 # 3'HC'
    df_discretized[c][df_discretized[c]==-10]=1
    df_discretized[c][df_discretized[c]==-20]=2
    df_discretized[c][df_discretized[c]==-30]=3
df_discretized = df_discretized.fillna(0)

C:\Users\prins\AppData\Local\Temp\ipykernel_19560\1012222020.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_discretized[c][df_discretized[c]<0.00001] = -1 # negative confidence (NC)
C:\Users\prins\AppData\Local\Temp\ipykernel_19560\1012222020.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_discretized[c][(df_discretized[c]<=q25) & (df_discretized[c]>0)] = -10 # 1 'LC'
C:\Users\prins\AppData\Local\Temp\ipykernel_19560\1012222020.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

In [2]:
from cobra.io import load_matlab_model, save_matlab_model
mod = load_matlab_model('C:/Users/prins/git/Human-GEM/model/Human-GEM.mat') # load HUMAN1 model (this takes really long!!)

In [3]:
conditions

Index(['MeanExpression_SchwannCells_1__VoigtEtAl2019_',
       'MeanExpression_SchwannCells_2__VoigtEtAl2019_',
       'MeanExpression_Melanocytes__VoigtEtAl2019_',
       'MeanExpression_EndothelialCells__VoigtEtAl2019_',
       'MeanExpression_SmoothMuscle__VoigtEtAl2019_',
       'MeanExpression_Fibroblasts__VoigtEtAl2019_',
       'MeanExpression_RPE__VoigtEtAl2019_',
       'MeanExpression_BCells__VoigtEtAl2019_',
       'MeanExpression_TCells_NKCells__VoigtEtAl2019_',
       'MeanExpression_monocytesAnd_orMacrophages__VoigtEtAl2019_',
       'MeanExpression_mastCells__VoigtEtAl2019_',
       'MeanExpression_RETINA__WhitmoreEtAl_2014_',
       'MeanExpression_RPE_CHOROID__WhitmoreEtAl_2014_',
       'MeanExpression_RodPhotoreceptors__LiangEtAl_2019_',
       'MeanExpression_AmacrineCells__LiangEtAl_2019_',
       'MeanExpression_BipolarCells__LiangEtAl_2019_',
       'MeanExpression_ConePhotoreceptors__LiangEtAl_2019_',
       'MeanExpression_HorizontalCells__LiangEtAl_2019_',
   

In [ ]:
from corda import reaction_confidence
from cobra import Configuration
from corda import CORDA
config = Configuration()
config.solver = "glpk"

def context_specific_model(mod, condition): 
    mod_c = mod.copy() # make deep copy of basis model 'mod' (Human1) for every condition
    mod_c.id = c
    gene_conf_dict = dict([(i,x) for i, x in zip(df_discretized.Ensembl, df_discretized[c])]) # gene confidences dictionary
    rxn_conf_dict = {rxn.id:reaction_confidence(rxn, gene_conf_dict) for rxn in mod.reactions} # reaction confidence dictionary
    mod_CORDA = CORDA(mod, rxn_conf_dict) 
    mod_CORDA.build()
    print(c)
    print(mod_CORDA)
    mod_c = mod_CORDA.cobra_model()
    return  mod_c

c = 'MeanExpression_RodPhotoreceptors__LiangEtAl_2019_'
mod_PRrod = context_specific_model(mod,c)
save_matlab_model(mod_PRrod, "C:/Users/prins/mod_PRrod.mat")
mod_PRrod = load_matlab_model("C:/Users/prins/mod_PRrod.mat")

In [ ]:
mod1

In [ ]:
from corda import reaction_confidence
from cobra import Configuration
from corda import CORDA
config = Configuration()
config.solver = "glpk"
i=0
models={} # empty dict
for c in conditions:
    mod_c = mod.copy() # make deep copy of basis model 'mod' (Human1) for every condition
    mod_c.id = c
    gene_conf_dict = dict([(i,x) for i, x in zip(df_discretized.Ensembl, df_discretized[c])]) # gene confidences dictionary
    rxn_conf_dict = {rxn.id:reaction_confidence(rxn, gene_conf_dict) for rxn in mod.reactions} # reaction confidence dictionary
    mod_CORDA = CORDA(mod, rxn_conf_dict) 
    mod_CORDA.build()
    print(c)
    print(mod_CORDA)
    mod_c = mod_CORDA.cobra_model()
    models[i]=mod_c
    i = i + 1

In [ ]:
from corda import reaction_confidence
c = conditions [0]
gene_conf_dict = dict([(i,x) for i, x in zip(df_discretized.Ensembl, df_discretized[c])]) # gene confidences dictionary
rxn_conf_dict = {rxn.id:reaction_confidence(rxn, gene_conf_dict) for rxn in mod.reactions} # reaction confidence dictionary